In [1]:
import numpy as np
import pandas as pd
import scipy as scp

## data import

In [ ]:
from astroquery.vizier import Vizier


VizOC = Vizier(catalog="J/A+A/659/A59/table2")
VizOC.ROW_LIMIT = -1

cluster = "Blanco_1"
OC_df = VizOC.query_constraints(Cluster=cluster).values()[0].to_pandas()

## building density profile

## fit different functions

## compare models

## HOORAY!!!